### +++ static +++

In [1]:
import opensim as osim
import os
import sys

from simFunctions import *
from subprocess import call
from stan_utils import subject_specific_isometric_force
from stan_utils import *
import pandas as pd

In [2]:
os.getcwd()

'd:\\Docs_ASUS\\WORK\\hip\\MotionStudy\\MotionData\\001'

# Scaling

This file uses scaling scheme that is already given in scaling_setting.xml.

For walking, the model is scaled following distances between external markers and muscle forces are scaled by a factor calculated with mass and height of the subject relative the mass and the height of the generic model.

In [3]:
# paths to and names of files

# <model_file>
path_to_generic_model = 'RajagopalModified_generic_copy.osim'

# <marker_set_file>
path_to_generic_marker_set = 'scaling_EXP_Markers_to_Scale_final_3.xml'

# path to experimental .trc file : <marker_file>
experimental_markers = r'D:\Docs_ASUS\WORK\hip\MotionStudy\MotionData\001\P01\Python\static\Static03\task.trc'
static_df = pd.read_csv(experimental_markers, delimiter='\t', skiprows=3, header=[0,1], index_col=0)

# <time_range>'
time_range = f' 0 {list(static_df.loc[static_df.shape[0], "Time"])[0] }'

# output model : <output_model_file>
output_model_file = 'scaled_model_P01.osim'

# <output_scale_file>
output_scale_file = 'output_scale_file_P01.txt'

mass_text = '52.4'

height_text = '1.6'

age_text = '33'

In [4]:
import xml.etree.ElementTree as ET

# parse the .xml with markers in body frames
scaling_tree=ET.parse("scaling_setting.xml")
scaling_root = scaling_tree.getroot()

for generic_model in scaling_root.iter('model_file'):
    generic_model.text = path_to_generic_model

for generic_marker_set in scaling_root.iter('marker_set_file'):
    generic_marker_set.text = path_to_generic_marker_set

for exp_markers in scaling_root.iter('marker_file'):
    exp_markers.text = experimental_markers

for time in scaling_root.iter('time_range'):
    time.text = time_range

for output in scaling_root.iter('output_model_file'):
    output.text = output_model_file

for scale in scaling_root.iter('output_scale_file'):
    scale.text = output_scale_file

for generic_marker_set in scaling_root.iter('marker_set_file'):
    generic_marker_set.text = path_to_generic_marker_set

for exp_markers in scaling_root.iter('marker_file'):
    exp_markers.text = experimental_markers

for time in scaling_root.iter('time_range'):
    time.text = time_range

for output in scaling_root.iter('output_model_file'):
    output.text = output_model_file

for scale in scaling_root.iter('output_scale_file'):
    scale.text = output_scale_file

In [5]:
for mass in scaling_root.iter('mass'):
	mass.text = mass_text

for height in scaling_root.iter('height'):
	height.text = height_text

for age in scaling_root.iter('age'):
	age.text = age_text

In [6]:
scaling_tree.write(os.path.join('scaling_setting_P01.xml'))

## 2.1 Scale model to height and weight

In [7]:
cmdprog = 'opensim-cmd'
cmdtool = 'run-tool'
cmdfile = 'scaling_setting_P01.xml'
cmdfull = [cmdprog, cmdtool, cmdfile]
rc = runProgram(cmdfull)

c:\Users\Ekatarina\miniconda3\envs\py38\lib\subprocess.py:848: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
  self.stdout = io.open(c2pread, 'rb', bufsize)


[info] Preparing to run ScaleTool.

[info] Processing subject Rajagopal-generic-scaled...

[info] Step 1: Loading generic model

[info] Loaded model RajagopalModifiedGeneric from file d:\Docs_ASUS\WORK\hip\MotionStudy\MotionData\001\RajagopalModified_generic_copy_with_joint_centers.osim

[info] Loading marker set from 'scaling_EXP_Markers_to_Scale_final_3.xml'.

[info] Updated markers in model RajagopalModifiedGeneric

[info] Step 2: Scaling generic model

[info] Loaded marker file D:\Docs_ASUS\WORK\hip\MotionStudy\MotionData\001\P01\Python\static\Static03\task.trc (40 markers, 445 frames)

[info] Measurement 'pelvis width'

[info] 	pair 0 (LASI, RASI): model = 0.249129, experimental = 0.222911

[info] 	pair 1 (LPSI, RPSI): model = 0.0880006, experimental = 0.0854253

[info] 	overall scale factor = 0.932748

[info] Measurement 'pelvis height'

[info] 	pair 0 (LASI, LMKNE): model = 0.502243, experimental = 0.458841

[info] 	pair 1 (RASI, RMKNE): model = 0.490209, experimental = 0.456198

<div class="alert alert-block alert-info">
<b>IN OPENSIM GUI</b> place centers of joints: 
    <li>torso_origin_in_pelvis</li>
    <li>pelvis_origin_in_ground</li>
    <li>femur_l_center_in_pelvis</li>
    <li>femur_r_center_in_pelvis</li>
    <li>knee_l_center_in_femur_l</li>
    <li>knee_r_center_in_femur_r</li>
    <li>patella_l_in_femur_l</li>
    <li>patella_r_in_femur_r</li>
    <li>ankle_l_tibia_l_offset</li>
    <li>ankle_r_tibia_r_offset</li>

</div>

## 2.2 Scale model muscle forces to height and weight for walking

generic_model = osim.Model('RajagopalModified_generic_copy.osim')
model=osim.Model(os.path.join('scaled_2.osim'))
scaled = scaleOptimalForceSubjectSpecific(generic_model, model, 1.77, 1.60)

pelvis_coordinates = model.getCoordinateSet().get('pelvis_tilt')
pelvis_coordinates.setRangeMax(4)
scaled.printToXML(os.path.join('forces_scaled_2.osim'))

## 2.3 Increase model muscle forces for running

This step is to create a second model, specifically for running. It is not necessary if the first model works well for both.

model=osim.Model(os.path.join('forces_scaled_2.osim'))
muscles=model.getMuscles()
muscles.getSize()

'glmax1_l', 'glmax1_r', 'glmax2_l', 'glmax2_r', 'glmax3_l', 'glmax3_r', 'glmed1_l', 'glmed1_r', 'glmed1_2', 'glmed2_r', 'glmed3_l', 'glmed3_r', 'glmin1_l', 'glmin1_r', 'glmin2_l', 'glmin2_r', 'glmin3_l', 'glmin3_r', 'semimem_l', 'semimem_r', 'semiten_l', 'semiten_r', 'soleus_l', 'soleus_r', 'tibpost_l', 'tibpost_r', 'vaslat_l', 'vaslat_r', 'vasmed_l', 'vasmed_r', 'vasint_l', 'vasint_r'

good_muscles = []

for i in range(0, muscles.getSize()):
    currentMuscle = muscles.get(i)
    if currentMuscle.getName() not in good_muscles:
        forceScaleFactor = 2
        currentMuscle.setMaxIsometricForce(forceScaleFactor * currentMuscle.getMaxIsometricForce() )        

pelvis_coordinates = model.getCoordinateSet().get('pelvis_tilt')
pelvis_coordinates.setRangeMax(4)
pelvis_coordinates.setRangeMin(-4)

model.printToXML(os.path.join('forces_scaled_2_running.osim'))